In [86]:
import csv # new 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup # conda install beautifulsoup4

In [104]:
url = "http://issue.cw.com.tw/ipad/201605_2000/list.jsp?orders=SER&sort=asc&inds={}&start={}"
filepath = "c:/test/F500-{}.csv"
cats = {"A": ("製造業", 1340),
        "B": ("服務業", 640),
        "C": ("金融業", 90)
       }

In [95]:
def getPage(inds, page):
    
    headers = {
        'User-agent' : 'none/ofyourbusiness', # readable header
    }

    req = urllib.request.Request(url = url.format(inds, page),
                                 headers = headers,
                                 method = 'POST')

    with urllib.request.urlopen(req) as f:
        data = b''
        buf = f.read(1024)
        data += buf
        while buf:
            buf = f.read(1024)
            data += buf
        #print(data.decode('utf-8'))
    
    return data

In [89]:
def getTitle(soup):
    title_list = []

    for th in soup.find_all('th'):
        title_list.append(th.text)

    years = title_list[0].split(" ")
    title_list.insert(0, years[0])
    title_list[1] = years[1]

    return title_list

In [90]:
def getRows(soup, csv_list):

    for tr in soup.find_all('tr'):
        if not tr.find("td", class_= "tr_sort"): # skip first row

            this_row = list()
            for td in tr.find_all("td"):
                #print(td.text)
                this_row.append(td.text.strip())
            if len(this_row) > 0:
                csv_list.append(this_row)

In [ ]:
# main function
for cat in cats:
    inds = cat
    name = cats[cat][0]
    max_item = cats[cat][1]
    
    csv_list = list()
    title_list = None

    for p in range(0, max_item+1, 10): # get 10 rows each page
        #print(inds, p)
        data = getPage(inds, p)
        soup = BeautifulSoup(data, 'html.parser')

        if not title_list:
            title_list = getTitle(soup)

        getRows(soup, csv_list)
    
    df = pd.DataFrame(csv_list, columns = title)
    df.to_csv(filepath.format(inds), index = False)

In [107]:
#df = pd.DataFrame(csv_list, columns = title)
#df # show last df data